In [21]:
import os
import zipfile
import torch
import requests
import json



In [31]:
url = '/Volumes/Elements/Organoides/Noyaux'
url_dest = '/Volumes/Elements/Organoides/DatasetINRIA_Organoids.zip'

In [32]:
# Leggi il token di accesso Zenodo dal file tokenZenodo.txt
with open('tokenZenodo.txt', 'r') as token_file:
    ACCESS_TOKEN = token_file.read().strip()


In [33]:
# Crea un archivio ZIP della cartella
def create_zip_archive(folder_path, zip_name=None):
    """
    Crea un archivio ZIP di una cartella mantenendo la struttura
    """
    if zip_name is None:
        zip_name = f"{os.path.basename(folder_path)}.zip"

    print(f"🗜️ Creando archivio ZIP: {zip_name}")

    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Mantiene la struttura delle cartelle nell'archivio
                arc_name = os.path.relpath(file_path, os.path.dirname(folder_path))
                zipf.write(file_path, arc_name)
                print(f"  📁 Aggiunto: {arc_name}")

    # Mostra dimensione dell'archivio
    zip_size = os.path.getsize(zip_name)
    print(f"✅ Archivio creato: {zip_name} ({zip_size/1024/1024:.2f} MB)")
    return zip_name

In [34]:

# Crea una nuova deposizione
def create_deposition():
    url = 'https://zenodo.org/api/deposit/depositions'
    headers = {"Content-Type": "application/json"}
    params = {'access_token': ACCESS_TOKEN}

    data = {
        'metadata': {
            'title': 'Dataset_INRIA',
            'upload_type': 'organoid tiff',
            'description': 'Dataset tiff dataset',
            'creators': [{'name': 'Dataset_INRIA_organoid'}]
        }
    }

    r = requests.post(url, params=params, data=json.dumps(data), headers=headers)
    return r.json()

# Carica il file
def upload_file(deposition_id, file_path):
    # Get bucket URL
    url = f'https://zenodo.org/api/deposit/depositions/{deposition_id}'
    params = {'access_token': ACCESS_TOKEN}
    r = requests.get(url, params=params)
    bucket_url = r.json()["links"]["bucket"]

    # Upload file
    filename = os.path.basename(file_path)
    with open(file_path, "rb") as fp:
        r = requests.put(f"{bucket_url}/{filename}",
                        data=fp,
                        params=params)
    return r.json()

# Pubblica il dataset
def publish_deposition(deposition_id):
    url = f'https://zenodo.org/api/deposit/depositions/{deposition_id}/actions/publish'
    params = {'access_token': ACCESS_TOKEN}
    r = requests.post(url, params=params)
    return r.json()


In [35]:
# Percorso della cartella da caricare
folder_to_upload = url  # Cambia con il percorso della tua cartella
zip_filename = url_dest  # Nome dell'archivio
zip_path = create_zip_archive(folder_to_upload, zip_filename)

🗜️ Creando archivio ZIP: /Volumes/Elements/Organoides/DatasetINRIA_Organoids.zip


OSError: [Errno 30] Read-only file system: '/Volumes/Elements/Organoides/DatasetINRIA_Organoids.zip'

In [ ]:
# Esegui l'upload
print("Creando deposizione...")
deposition = create_deposition()
deposition_id = deposition['id']

print(f"Caricando file... (ID: {deposition_id})")
upload_result = upload_file(deposition_id, zip_filename)

print("Pubblicando dataset...")
publication = publish_deposition(deposition_id)

print(f"Dataset pubblicato! DOI: {publication['doi']}")
print(f"URL: {publication['links']['record_html']}")
